<a href="https://colab.research.google.com/github/swapnilmn/Assignment_1-CS6910/blob/master/Question_2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.

Your code should be flexible such that it is easy to change the number of hidden layers and the number of neurons in each hidden layer.

# Libraries 

In [1]:
from keras.datasets import fashion_mnist, mnist
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from sklearn.model_selection import train_test_split 
import pickle
import pandas as pd
import os
from tqdm import tqdm_notebook 

In [12]:
class OneHotEncoder_from_scratch:
    
    def __init__(self):
        self.categories = None
    def fit(self, X):
        self.categories =[]
        for i in range(X.shape[1]):
            feature_categories =list(set(X[:, i]))
            self.categories.append(feature_categories)
            
    def transform(self, X):
        one_hot_vector = []

        for i in range(X.shape[0]):
            one_hot_row = []
            for j in range(X.shape[1]):

                category_index = self.categories[j].index(X[i, j])
                category_one_hot =[0] *len(self.categories[j])
                category_one_hot[category_index] = 1

                one_hot_row.extend(category_one_hot)
            one_hot_vector.append(one_hot_row)
        return np.array(one_hot_vector)

X = np.array([[1],[2]]) 
enc = OneHotEncoder_from_scratch()
enc.fit(X)
enc.transform(X)

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
enc.fit(train_labels.reshape(-1, 1))
enc.transform(train_labels.reshape(-1, 1))
#train_labels.reshape(-1, 1)

array([[0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
dataset = 'fashion_mnist'
if dataset == 'fashion_mnist':
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
elif dataset == 'mnist':
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
else:
    raise ValueError('Invalid dataset name')
X_train, X_val, Y_train, Y_val = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)
train_input = []
for i in range(len(X_train)):
    train_input.append(list(np.concatenate(X_train[i]).flat))

val_input = []
for i in range(len(X_val)):
    val_input.append(list(np.concatenate(X_val[i]).flat))

test_input = []
for i in range(len(test_images)):
    test_input.append(list(np.concatenate(test_images[i]).flat))
Y_train = np.array(Y_train)
Y_val = np.array(Y_val)
Y_test = np.array(test_labels)

X_train = np.array(train_input) / 255.0
X_test = np.array(test_input) / 255.0
X_val = np.array(val_input) / 255.0

enc = OneHotEncoder_from_scratch()
enc.fit(Y_train.reshape(-1, 1))
Y_train = enc.transform(Y_train.reshape(-1, 1))
Y_val = enc.transform(Y_val.reshape(-1, 1))
Y_test = enc.transform(Y_test.reshape(-1, 1))

print(Y_train.shape, Y_val.shape, Y_test.shape)
print(X_train.shape, X_val.shape, X_test.shape)

(54000, 10) (6000, 10) (10000, 10)
(54000, 784) (6000, 784) (10000, 784)


In [15]:
# we will set a configuartion dictionary for future use for wandb seerp
# we will take input from that configuration dict

config = {'hidden_layers_size' : [32, 64, 128], # We can modify numbers of neurons each layers, parallaly it will modify number of layers
          'pre_activation_function': 'sigmoid',
          'weight_initialization_method': 'xavier',
          }
# Now we will dynamically add no of hidden layers key 
config['hidden_layers_no'] = len(config['hidden_layers_size'])


In [20]:
class FFNetwork:
  def __init__(self, X, Y,
               epochs = 100, 
               hidden_layer_count = 4,
               hidden_layers =  [32, 64, 128, 256],
               learning_rate = 0.001,
               batch_size = 32,
               activation='tanh',
               weight_init='random',
               loss = 'MSE',
               weight_decay = 0):
    
    self.inputs =X.shape[1] # Number of inputs
    self.outputs= Y.shape[1] # Number of outputs
    self.epochs = epochs
    self.hidden_layers = hidden_layer_count  # Number of hidden layers 
    self.network_size= [self.inputs] + hidden_layers +[self.outputs] # input layer + hidden layers + output layers
    self.learning_rate = learning_rate
    self.batch_size = batch_size
    self.weights={} # It will create dictionary for weights and biases
    self.weights_h = []
    self.num_classes = Y.shape[1]
    self.weight_init = weight_init
    self.activation_function = activation
    self.loss_function = loss
    self.lambd = 0
    np.random.seed(0)  # We will set seed value so that it will generate same random numebers every time

    self.grad_derivatice={}
    self.update_weights={}
    self.prev_update_weights={}
    for i in range(1,self.hidden_layers+1):
      self.update_weights['vw'+str(i)]=0
      self.update_weights['vb'+str(i)]=0
      self.update_weights['mb'+str(i)]=0
      self.update_weights['mw'+str(i)]=0
      self.prev_update_weights['vw'+str(i)]=0
      self.prev_update_weights['vb'+str(i)]=0
    
    if self.weight_init == 'random':
      for i in range(1,self.hidden_layers+2):
        self.weights['W'+str(i)] = np.random.randn(self.network_size[i-1],self.network_size[i])*0.1
        self.weights['B'+str(i)] =np.zeros((1,self.network_size[i]))
    
    elif self.weight_init == 'Xavier':
      for i in range(1,self.hidden_layers+2):
        self.weights['W'+str(i)]=np.random.randn(self.network_size[i-1],self.network_size[i])*np.sqrt(1/self.network_size[i-1])
        self.weights['B'+str(i)] = np.zeros((1,self.network_size[i]))

  
  def forward_activation(self, X): 
    if self.activation_function =='sigmoid':
      return 1.0/(1.0 +np.exp(-X))
    elif self.activation_function== 'tanh':
      return  np.tanh(X)
    elif self.activation_function =='Relu':
      return  np.maximum(0,X)


  def grad_activation(self, X):
    if self.activation_function =='sigmoid':
      return X*(1-X) 
    elif self.activation_function== 'tanh':
      return (1-np.square(X))
    elif  self.activation_function == 'Relu':
      return 1.0*(X>0)

  def softmax(self, X):
    exps =np.exp(X -np.max(X, axis=1,keepdims=True))
    return  exps/np.sum(exps,axis=1, keepdims=True)
  
  def forward_pass(self,X, weights=None):
    if weights is None:
        weights =self.weights
    self.z = {}
    self.h ={}
    self.h[0] =X
    for i in range(self.hidden_layers):
        self.z[i+1]=np.matmul(self.h[i], weights[f'W{i+1}']) + weights[f'B{i+1}']
        self.h[i+1] =self.forward_activation(self.z[i+1])
    self.z[self.hidden_layers+1] = np.matmul(self.h[self.hidden_layers],weights[f'W{self.hidden_layers+1}']) +weights[f'B{self.hidden_layers+1}']
    self.h[self.hidden_layers+1] =self.softmax(self.z[self.hidden_layers+1])
    return self.h[self.hidden_layers+1]   
  
model = FFNetwork(X_train, Y_train,
                  epochs = 1, 
                  hidden_layer_count = 4,
                  hidden_layers =  [32, 64, 128, 256],
                  learning_rate = 0.001,
                  batch_size = 32,
                  activation='Relu',
                  weight_init='random',
                  loss = 'CE')
model.forward_pass(X_train)[0]

array([0.10439453, 0.10853244, 0.09621989, 0.09402379, 0.09662353,
       0.10174432, 0.09815351, 0.10075173, 0.10590278, 0.09365347])